In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics

%matplotlib inline

In [ ]:
df = pd.read_csv("deliveries_updated_mens_ipl_upto_2024 - deliveries_updated_mens_ipl_upto_2024.csv")
df.head()

In [21]:
df.columns

Index(['matchId', 'inning', 'over_ball', 'over', 'ball', 'batting_team',
       'bowling_team', 'batsman', 'non_striker', 'bowler', 'batsman_runs',
       'extras', 'isWide', 'isNoBall', 'Byes', 'LegByes', 'Penalty',
       'dismissal_kind', 'player_dismissed', 'date'],
      dtype='object')

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
[i for i in df.columns if df[i].isnull().sum()>0]


In [20]:
filled_df=df

fillwith0=["isWide","isNoBall","Byes","LegByes","Penalty"]
filled_df[fillwith0]=filled_df[fillwith0].fillna(0)

fillwithStr = ["dismissal_kind","player_dismissed"]
filled_df[fillwithStr]=filled_df[fillwithStr].fillna("No Dismissal")


filled_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   matchId           260920 non-null  int64  
 1   inning            260920 non-null  int64  
 2   over_ball         260920 non-null  float64
 3   over              260920 non-null  int64  
 4   ball              260920 non-null  int64  
 5   batting_team      260920 non-null  object 
 6   bowling_team      260920 non-null  object 
 7   batsman           260920 non-null  object 
 8   non_striker       260920 non-null  object 
 9   bowler            260920 non-null  object 
 10  batsman_runs      260920 non-null  int64  
 11  extras            260920 non-null  int64  
 12  isWide            260920 non-null  float64
 13  isNoBall          260920 non-null  float64
 14  Byes              260920 non-null  float64
 15  LegByes           260920 non-null  float64
 16  Penalty           26

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,date
0,335982,1,0.1,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,0.0,0.0,0.0,1.0,0.0,No Dismissal,No Dismissal,2008-04-18
1,335982,1,0.2,0,2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,0.0,0.0,0.0,0.0,0.0,No Dismissal,No Dismissal,2008-04-18
2,335982,1,0.3,0,3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,0.0,0.0,0.0,0.0,No Dismissal,No Dismissal,2008-04-18
3,335982,1,0.4,0,4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,0.0,0.0,0.0,0.0,0.0,No Dismissal,No Dismissal,2008-04-18
4,335982,1,0.5,0,5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,0.0,0.0,0.0,0.0,0.0,No Dismissal,No Dismissal,2008-04-18
5,335982,1,0.6,0,6,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,0.0,0.0,0.0,0.0,0.0,No Dismissal,No Dismissal,2008-04-18
6,335982,1,0.7,0,7,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,0.0,0.0,0.0,1.0,0.0,No Dismissal,No Dismissal,2008-04-18
7,335982,1,1.1,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,Z Khan,0,0,0.0,0.0,0.0,0.0,0.0,No Dismissal,No Dismissal,2008-04-18
8,335982,1,1.2,1,2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,Z Khan,4,0,0.0,0.0,0.0,0.0,0.0,No Dismissal,No Dismissal,2008-04-18
9,335982,1,1.3,1,3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,Z Khan,4,0,0.0,0.0,0.0,0.0,0.0,No Dismissal,No Dismissal,2008-04-18


In [ ]:
# sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
# df.dtypes